# Function Call

# 準備

In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
import openai

# 環境変数の読み込み
load_dotenv()
openai.api_key = os.environ['API_KEY']
model_name = "gpt-3.5-turbo"

# シンプルな例

In [2]:
import json

# 検索結果を返す関数の作成
def get_search_result(keyword, when, unit="d"):
    result = {
        "result": [
            {"title": "1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました"},
            {"title": "1929年（昭和4年）八重洲口が開設されました"},
            {"title": "2014年（平成26年）東京駅開業100周年を迎えました"},
        ]
    }

    return json.dumps(result)

In [3]:
# 関数情報の作成
functions=[
    {
        "name": "get_search_result",
        "description": "指定したキーワードの検索結果を取得する",
        "parameters": {
            "type": "object",
            "properties": {
                "keyword": {
                    "type": "string",
                    "description": "キーワード",
                },
                "when": {
                    "type": "number",
                    "description": "日付や時間の範囲"
                },
                "unit": {
                    "type": "string",
                    "enum": ["d", "h", "y"]
                },
            },
            "required": ["keyword"],
        },
    }
]

In [4]:
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"

response = openai.ChatCompletion.create(
    model=model_name,
    messages=[
        {"role": "user", "content": question},
    ],
    functions=functions,
    function_call="auto",
)

In [5]:
response

<OpenAIObject chat.completion id=chatcmpl-7XgYHkrvWlECC4bpVhFbrtioPLZn0 at 0x1c8f3722750> JSON: {
  "id": "chatcmpl-7XgYHkrvWlECC4bpVhFbrtioPLZn0",
  "object": "chat.completion",
  "created": 1688262749,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_search_result",
          "arguments": "{\n  \"keyword\": \"\u6771\u4eac\u99c5\u306e\u30a4\u30d9\u30f3\u30c8\",\n  \"when\": 1,\n  \"unit\": \"m\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 135,
    "completion_tokens": 40,
    "total_tokens": 175
  }
}

In [6]:
response["choices"][0]["message"]["function_call"]["arguments"]

'{\n  "keyword": "東京駅のイベント",\n  "when": 1,\n  "unit": "m"\n}'

In [7]:
# 言語モデルからの回答メッセージを取得
message = response["choices"][0]["message"]

# モデルが関数呼出と判断した
if message.get("function_call"):
    # 関数名の取得
    function_name = message["function_call"]["name"]
    print(f"関数名：{function_name}")
    # 引数の取得
    arguments = json.loads(message["function_call"]["arguments"])
    print(f"引数：{arguments}")

    # 関数名で実行する関数を判断
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )
    print(f"関数の実行結果：{function_response}")

    # 関数の実行結果をmessagesに加えて言語モデルを呼出
    response2 = openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {"role": "user", "content": question},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print(response2.choices[0]["message"]["content"].strip())

else:
    # 関数呼び出しでなければ、単に言語モデルからの回答を出力
    print(response.choices[0]["message"]["content"].strip())

関数名：get_search_result
引数：{'keyword': '東京駅のイベント', 'when': 1, 'unit': 'm'}
関数の実行結果：{"result": [{"title": "1914\u5e74\uff08\u5927\u6b633\u5e74\uff09\u4e2d\u592e\u505c\u8eca\u5834\u6539\u3081\u300c\u6771\u4eac\u99c5\u300d\u3068\u3057\u3066\u55b6\u696d\u3092\u958b\u59cb\u3057\u307e\u3057\u305f"}, {"title": "1929\u5e74\uff08\u662d\u548c4\u5e74\uff09\u516b\u91cd\u6d32\u53e3\u304c\u958b\u8a2d\u3055\u308c\u307e\u3057\u305f"}, {"title": "2014\u5e74\uff08\u5e73\u621026\u5e74\uff09\u6771\u4eac\u99c5\u958b\u696d100\u5468\u5e74\u3092\u8fce\u3048\u307e\u3057\u305f"}]}
最近の1ヶ月以内の検索結果によると、東京駅のイベントに関して以下の内容があります。

1. 1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました。

2. 1929年（昭和4年）八重洲口が開設されました。

3. 2014年（平成26年）東京駅開業100周年を祝うイベントが行われました。

詳細な内容や開催情報はわかりませんので、詳細を知りたい場合はイベントの公式ウェブサイトや関連情報をご確認ください。
